## Prepaparation

In [2]:
import pandas as pd
import numpy as np

#model
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

#other
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import math



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Loading datasets

In [3]:
print('loading dataset...')
tr = pd.read_csv("./training_data.csv")
#te = pd.read_csv("./testing_data.csv")


tr_label = tr['Next_Premium']
tr_feature = tr
drop_feature = ['Next_Premium','Prior_Policy_Number','nequipment9','Vehicle_Make_and_Model1','Distribution_Channel','Accident_Date','Claim_Number']
for fe in drop_feature:
    tr_feature = tr_feature.drop(fe, axis=1)

tr_x, te_x, tr_y, te_y = train_test_split(tr_feature, tr_label, test_size=0.2, random_state=2018)

tr_y = np.reshape(tr_y, (-1, 1))
te_y = np.reshape(te_y, (-1, 1))
scaler_x = StandardScaler().fit(tr_x)
scalerY = StandardScaler()

tr_x = scaler_x.transform(tr_x)
te_x = scaler_x.transform(te_x)
#tr_y = scaler_y.transform(tr_y)
#te_y = scaler_y.transform(te_y)

print('loading complete')

loading dataset...


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


loading complete


## Standard ML model

## SVR

In [4]:
model_SVR = SVR()
#model_SVR_params = {'C':[val*0.01 for val in range(1, 10)]}
model_SVR_params = {'C':[0.1, 0.05, 0.01, 0.5]}

## Random forests regression

In [5]:
model_RF = RandomForestRegressor()
model_RF_params = {'n_jobs' : [6], 'random_state' : [2018], 'n_estimators' : [val for val in range(1, 10)], 'max_depth' : [val for val in range(1, 10)]}

## Elastic net regression

In [6]:
model_EN = linear_model.ElasticNetCV(random_state=2018, n_jobs=10)
model_EN_params = {'l1_ratio': [.1, .5, .7, .9, .95, .99, 1]}
#model_EN_params = {}

## Linear regression

In [7]:
model_LR = linear_model.LinearRegression()
model_LR_params = {'n_jobs': [10]
}

## XGB

In [24]:
model_xgb_params =  {'booster': ['gblinear'],
                     'colsample_bytree': [0.2],
                     'gamma': [0.01],
                     'learning_rate': [0.5],
                     'max_depth': [val for val in range(3, 8)],
                     'min_child_weight': [val for val in range(1, 6)],
                     'n_estimators': [191],
                     'objective': ['reg:linear'],
                     'random_state': [2018],
                     'silent': [1],
                     'subsample': [0.8]}
model_xgb = xgb.XGBRegressor(n_jobs=10)

## Training

In [25]:
#model_list = [model_SVR, model_RF, model_EN, model_xgb]
model_list = [model_xgb]
params = {model_SVR:model_SVR_params, model_RF:model_RF_params, model_EN:model_EN_params, model_xgb:model_xgb_params, model_LR:model_LR_params}

print('predicting...')
for model in model_list:
    #sc = cross_val_score(model, tr_feature, tr_label, scoring='neg_mean_absolute_error', cv=2)
    grid = GridSearchCV(model, params[model], verbose=True)
    grid.fit(tr_x, tr_y)
    pred = grid.predict(te_x)
    #pred = scaler_y.inverse_transform(pred)
    #te_y = scaler_y.inverse_transform(te_y)
    sc = mean_absolute_error(te_y, pred)
    print('')
    print(str(model) + ' / score: ' + str(abs(np.average(sc))))

predicting...
Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 13.0min finished



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=10, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1) / score: 2515.679182369497


## Deep learning

In [ ]:
#prepaparation
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 1 # 設定使用記憶體的％數
# set_session(tf.Session(config=config))

def layer(output_dim, input_dim, input, activation = None):
    with tf.name_scope('weight'):
        W = tf.Variable(tf.random_normal([input_dim, output_dim]))
    
    with tf.name_scope('bias'):
        b = tf.Variable(tf.random_normal([1, output_dim]))
    
    output = tf.matmul(input, W) + b
    
    if activation:
        output = activation(output)
    
    return output


with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, len(list(tr_x[0]))], name='feature')
    y = tf.placeholder(tf.float32, [None, 1], name='label')

with tf.name_scope('Weight'):
    W = tf.Variable(tf.truncated_normal([len(list(tr_x[0])), 1], stddev=0.1), name='Weight')

with tf.name_scope('bias'):
    b = tf.Variable(tf.constant(0.1, shape=[1]), name='bias')

with tf.name_scope('Layer'):
    l1 = layer(350, len(list(tr_x[0])), x, activation=None)
    h1 = layer(500, 350, l1, activation=tf.nn.relu)
    h2 = layer(600, 500, h1, activation=tf.nn.relu)
    h3 = layer(400, 600, h2, activation=tf.nn.relu)
    h4 = layer(200, 400, h3, activation=tf.nn.relu)
    y_pred = layer(1, 200, h4, activation=None)

with tf.name_scope('loss'):
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y, y_pred))))
    tf.summary.scalar('loss', loss)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

merge = tf.summary.merge_all()
saver = tf.train.Saver()

with tf.Session(config=tf.ConfigProto(log_device_placement = True, allow_soft_placement = True)) as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('Lien_dataset/tensorflow_info/train', sess.graph)
    test_writer = tf.summary.FileWriter('Lien_dataset/tensorflow_info/test', sess.graph)
    batch_size = 20
    batch_num = math.ceil(len(tr_y) / batch_size)

    for epoch in range(80):
        for batch_i in range(batch_num):
            try:
                batch_xs = tr_x[batch_i*batch_size: (batch_i+1)*batch_size]
                batch_ys = tr_y[batch_i*batch_size: (batch_i+1)*batch_size]
                
            except:
                batch_xs = tr_x[batch_i*batch_size:]
                batch_ys = tr_y[batch_i*batch_size:]

            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})

        sc = mean_absolute_error(te_y, y_pred.eval(feed_dict = {x:te_x}))
        print('Epoch: ' + str(epoch) + ' score: ' + str(sc))
    
    #saver.save(sess, 'Lien_dataset/Model_saver/FC3Layer_2087.ckpt')
            
        
        


## Test data prediction

In [ ]:
def layer(output_dim, input_dim, input, activation = None):
    with tf.name_scope('weight'):
        W = tf.Variable(tf.random_normal([input_dim, output_dim]))
    
    with tf.name_scope('bias'):
        b = tf.Variable(tf.random_normal([1, output_dim]))
    
    output = tf.matmul(input, W) + b
    
    if activation:
        output = activation(output)
    
    return output


with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, len(list(tr_x[0]))], name='feature')
    y = tf.placeholder(tf.float32, [None, 1], name='label')

with tf.name_scope('Weight'):
    W = tf.Variable(tf.truncated_normal([len(list(tr_x[0])), 1], stddev=0.1), name='Weight')

with tf.name_scope('bias'):
    b = tf.Variable(tf.constant(0.1, shape=[1]), name='bias')

with tf.name_scope('Layer'):
    l1 = layer(350, len(list(tr_x[0])), x, activation=None)
    h1 = layer(500, 350, l1, activation=tf.nn.relu)
    #h2 = layer(400, 500, h1, activation=None)
    y_pred = layer(1, 500, h1, activation=None)

saver = tf.train.Saver()
#tf.reset_default_graph()
    
test_res = pd.read_csv("./testing_data.csv")
drop_feature = ['Next_Premium','Prior_Policy_Number','nequipment9','Vehicle_Make_and_Model1','Distribution_Channel','Accident_Date','Claim_Number']
for fe in drop_feature:
    test_res = test_res.drop(fe, axis=1)
    
#predicting
with tf.Session() as sess:
    
    
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'Lien_dataset/Model_saver/FC3Layer_2087.ckpt')
    print('predicting')
    test_feature = scaler_x.transform(test_res)
    batch_size = 1
    batch_num = math.ceil(len(tr_y) / batch_size)
    pred = []

    for batch_i in range(batch_num):
        if batch_i % 100 == 0:
            print(str(batch_i) + '/' + str(batch_num))
        try:
            batch_xs = test_feature[batch_i*batch_size: (batch_i+1)*batch_size]

        except:
            batch_xs = test_feature[batch_i*batch_size:]
        res = y_pred.eval(feed_dict = {x:batch_xs})
        pred.append(res.reshape(-1).tolist())
    
    pred = scaler_y.inverse_transform(pred)
    pred = np.maximum(pred, 0)
    

    print('csv writing')
    #test file writing
    test_data_file = pd.read_csv("./testing-set.csv")
    test_data_file['Next_Premium'] = pred

    test_data_file

    test_data_file.to_csv('Lien_dataset/lien_test_result_0901.csv', index=False)


## Test data prediction

In [ ]:
test_res = pd.read_csv("./testing_data.csv")
drop_feature = ['Next_Premium','Prior_Policy_Number','nequipment9','Vehicle_Make_and_Model1','Distribution_Channel','Accident_Date','Claim_Number']
for fe in drop_feature:
    test_res = test_res.drop(fe, axis=1)
    
#predicting
print('predicting')
test_feature = scaler_x.transform(test_res)
pred =  grid.predict(test_feature)
pred = np.maximum(pred, 0)
    

print('csv writing')
#test file writing
test_data_file = pd.read_csv("./testing-set.csv")
test_data_file['Next_Premium'] = pred

test_data_file

test_data_file.to_csv('Lien_dataset/lien_test_result_0829.csv', index=False)
    

## Model saver

In [ ]:
from sklearn.externals import joblib

#joblib.dump(grid, 'Lien_dataset/Model_saver/xgb_0827.pkl')

model = joblib.load('Lien_dataset/Model_saver/xgb_0827.pkl')
model.best_params_

In [ ]:
pred = grid.predict(te_x)

pred = scaler_y.inverse_transform(pred)


In [17]:
grid.best_params_

{'booster': 'gblinear',
 'colsample_bytree': 0.2,
 'gamma': 0.1,
 'learning_rate': 0.01,
 'max_depth': 7,
 'min_child_weight': 4,
 'n_estimators': 191,
 'objective': 'reg:linear',
 'random_state': 2018,
 'silent': 1,
 'subsample': 0.7}

In [27]:
sc = mean_absolute_error(te_y, pred)
print(sc)

2515.679182369497
